In [1]:
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)
import copy
import h5py
import numpy as np
import matplotlib
import re

import matplotlib.pyplot as plt
from dedalus.extras import plot_tools

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize

import os
from os import listdir

In [2]:
# Parameters
Lx, Ly, Lz = 4,4,1
Nx, Ny, Nz = 128, 128, 32

Ra_D = -1.24e5
Prandtl = 0.7
N_s2 = 2

D_0 = 0
D_H = 1
M_0 = 0
M_H = -1

dealias = 3/2
stop_sim_time = 1000
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

# %%
# Bases
coords = d3.CartesianCoordinates('x','y', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
ybasis = d3.RealFourier(coords['y'], size=Ny, bounds=(0, Ly), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)

# %%
# Fields
p = dist.Field(name='p', bases=(xbasis,ybasis,zbasis))
D = dist.Field(name='D', bases=(xbasis,ybasis,zbasis))
M = dist.Field(name='M', bases=(xbasis,ybasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,ybasis,zbasis))
Z = dist.Field(name='Z', bases=zbasis)
tau_p = dist.Field(name='tau_p')
tau_B1 = dist.Field(name='tau_B1', bases=(xbasis,ybasis))
tau_B2 = dist.Field(name='tau_B2', bases=(xbasis,ybasis))
tau_D1 = dist.Field(name='tau_D1', bases=(xbasis,ybasis))
tau_D2 = dist.Field(name='tau_D2', bases=(xbasis,ybasis))
tau_M1 = dist.Field(name='tau_M1', bases=(xbasis,ybasis))
tau_M2 = dist.Field(name='tau_M2', bases=(xbasis,ybasis))
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=(xbasis,ybasis))
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=(xbasis,ybasis))

# Substitutions
kappa = (Ra_D * Prandtl/((D_0-D_H)*Lz**3))**(-1/2)
nu = (Ra_D / (Prandtl*(D_0-D_H)*Lz**3))**(-1/2)

      
#Kuo_Bretherton Equilibrium

#Ra_M
Ra_M = Ra_D*(M_0-M_H)/(D_0-D_H)
G_D=(D_0-D_H)/Lz
G_M=(M_0-M_H)/Lz
Ra_BV=N_s2*Lz**4/(nu*kappa)
print(Ra_M)
print(Ra_BV)

x,y,z = dist.local_grids(xbasis,ybasis,zbasis)
Z['g']=z
Z.change_scales(3/2)

ex,ey,ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)

B_op = (np.absolute(D - M - N_s2*Z)+ M + D - N_s2*Z)/2

LqW = (np.absolute(M-D+N_s2*Z)+ (M-D+N_s2*Z))/2

Max = lambda A,B: (abs(A-N_s2*Z-B)+A-N_s2*Z+B)/2
eva = lambda A: A.evaluate()

dz= lambda A: d3.Differentiate(A, coords['z'])
dx= lambda A: d3.Differentiate(A, coords['x'])
dy= lambda A: d3.Differentiate(A, coords['y'])

ux=u@ex
uy=u@ey
uz=u@ez

ux2=ux*ux
uy2=uy*uy
uz2=uz*uz

grad_u = d3.grad(u) + ez*lift(tau_u1) # First-order reduction
grad_M = d3.grad(M) + ez*lift(tau_M1) # First-order reduction
grad_D = d3.grad(D) + ez*lift(tau_D1) # First-order reduction

124000.0
248000.00000000003


In [3]:
folder_dir = "analysis"

file_paths = [os.path.join(folder_dir, file) for file in listdir(folder_dir) if os.path.isfile(os.path.join(folder_dir, file)) and file.endswith('.h5')]
#sort by the number in the file name
file_paths.sort(key=lambda f: int(re.sub('\D', '', f)))
print(file_paths)

['analysis/analysis_s1.h5', 'analysis/analysis_s2.h5', 'analysis/analysis_s3.h5', 'analysis/analysis_s4.h5']


In [9]:
if not os.path.exists('liquid water'):    
    os.mkdir('liquid water')
n=0
for file in file_paths:
    with h5py.File(file, mode='r') as file:
        st = file['scales/sim_time']
        simtime = np.array(st)
        print(len(simtime))
        liq_wat = file['tasks']['integ LqW by Z']
        print(np.shape(liq_wat))
        print(liq_wat[0])
        liq_wat2 = liq_wat[:,:,:,0]
        print(np.shape(liq_wat2))
        print(liq_wat2[0])
        break

1000
(1000, 128, 128, 1)
[[[9.04540081e-05]
  [4.51537035e-05]
  [1.29961558e-04]
  ...
  [1.12713890e-05]
  [1.04662025e-04]
  [7.85281385e-05]]

 [[6.06424219e-05]
  [6.34252770e-05]
  [9.50870542e-05]
  ...
  [1.46046694e-04]
  [9.50682784e-05]
  [8.62078376e-05]]

 [[1.11264390e-04]
  [7.16757140e-05]
  [7.53728120e-05]
  ...
  [1.57312961e-04]
  [5.11121783e-05]
  [6.86037206e-05]]

 ...

 [[8.33807662e-05]
  [3.52687542e-05]
  [7.81668851e-05]
  ...
  [9.86865244e-05]
  [9.62693754e-05]
  [8.26305750e-05]]

 [[1.19123057e-04]
  [1.07574071e-04]
  [1.02691046e-04]
  ...
  [1.16491073e-04]
  [1.10457795e-04]
  [1.00690416e-04]]

 [[9.95471457e-05]
  [1.10206717e-04]
  [1.17263141e-04]
  ...
  [6.01245989e-05]
  [9.57646763e-05]
  [6.44617439e-05]]]
(1000, 128, 128)
[[9.04540081e-05 4.51537035e-05 1.29961558e-04 ... 1.12713890e-05
  1.04662025e-04 7.85281385e-05]
 [6.06424219e-05 6.34252770e-05 9.50870542e-05 ... 1.46046694e-04
  9.50682784e-05 8.62078376e-05]
 [1.11264390e-04 7.167